In [1]:
from model_predict import LLMPredictions
from dotenv import load_dotenv
from prompts import system_prompt
from openai import OpenAI
import base64
import openai
import os
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
from prompts import (
    intro_prompt,
    few_shot_prompt,
    final_prediction_prompt,
    system_prompt,
)
from helpers_functions import StockWizzard
import json
import ast
from helpers_functions import merge_df
import mlflow
import logging
import shutil

# https://www.mlflow.org/docs/latest/model-registry.html#adding-an-mlflow-model-to-the-model-registry
load_dotenv()

secret_key = os.getenv("openai_general_key")
secret_key_alpha = os.getenv("secret_key_alpha")

models = ["gpt-4o-mini", "gpt-4o-2024-05-13"]
ticker = "ON"
indicators = ""
timeframe = "weekly"
model = models[0]
train_window = 7
target_window = 1
few_shot_train_set = 3
test_set = 3


In [2]:
event = {
    "ticker": ticker,
    "model": models[0],
    "test_set": test_set,
    "indicators": "",
    "train_window": train_window,
    "target_window": target_window,
    "few_shot_train_set": few_shot_train_set,
    "timeframe": timeframe,
    "main_path": r"C:\Users\z0040jeb\Desktop\predictions",
}


## Save as mlflow model
# if os.path.exists("test_local"):
#     shutil.rmtree("test_local")  # save local

# mlflow.pyfunc.save_model(
#     "test_local",
#     python_model=LLMPredictions(),
#     code_path=["./helpers_functions.py", "./prompts.py"],
# )

## Register the model in mlflow registry
import mlflow

with mlflow.start_run() as run:
    if os.path.exists("local_model"):
        shutil.rmtree("local_model")  # save local

    mlflow.pyfunc.log_model(
        artifact_path="local_model",
        python_model=LLMPredictions(),
        code_path=["./helpers_functions.py", "./prompts.py"],
    )
    
    # Optionally register the model
    model_uri = f"runs:/{run.info.run_id}/model"
    mlflow.register_model(model_uri, "model_predictions")



c:\Users\z0040jeb\AppData\Local\miniconda3\envs\stock_predictor\Lib\site-packages\mlflow\pyfunc\__init__.py:2474: UserWarning: The `code_path` argument is replaced by `code_paths` and is deprecated as of MLflow version 2.12.0. This argument will be removed in a future release of MLflow.
  warnings.warn(
2024/10/13 10:25:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'model_predictions' already exists. Creating a new version of this model...
Created version '4' of model 'model_predictions'.


In [7]:
# from mlflow.models import validate_serving_input

# model_uri = 'runs:/0526a920629441109bde3d18d5553f7a/local_model'

# # The logged model does not contain an input_example.
# # Manually generate a serving payload to verify your model prior to deployment.
# from mlflow.models import convert_input_example_to_serving_input

# # Define INPUT_EXAMPLE via assignment with your own input example to the model
# # A valid input example is a data instance suitable for pyfunc prediction
# serving_payload = convert_input_example_to_serving_input(event)

# # Validate the serving payload works on the model
# validate_serving_input(model_uri, serving_payload)

c:\Users\z0040jeb\AppData\Local\miniconda3\envs\stock_predictor\Lib\site-packages\mlflow\models\utils.py:483: FutureWarning: Since MLflow 2.16.0, we no longer convert dictionary input example to pandas Dataframe, and directly save it as a json object. If the model expects a pandas DataFrame input instead, please pass the pandas DataFrame as input example directly.
  example = _Example(input_example)
c:\Users\z0040jeb\AppData\Local\miniconda3\envs\stock_predictor\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TypeError: LLMPredictions.predict() takes 2 positional arguments but 3 were given

In [3]:
import mlflow.pyfunc

# Load the model from the MLflow registry
model_name = "local_model"
model_version = 4  # You can specify the version or use a stage like 'Production'

# Load the latest model registered under this name and version
model_uri = 'runs:/0526a920629441109bde3d18d5553f7a/local_model'
loaded_model = mlflow.pyfunc.load_model(model_uri)

In [4]:
loaded_model.predict(event)

Directory created: C:\Users\z0040jeb\Desktop\predictions\chart_images/ON
Directory created: C:\Users\z0040jeb\Desktop\predictions\chart_images/chart_images/ON
Subfolder 'C:\Users\z0040jeb\Desktop\predictions\chart_images/ON\train' does not exist.
Subfolder 'C:\Users\z0040jeb\Desktop\predictions\chart_images/ON\test' does not exist.
Subfolder 'C:\Users\z0040jeb\Desktop\predictions\chart_images/ON\final_chart' does not exist.
target window :  1
Saved chart and label for window 0 to chart_images/ON/train/iteration_1
Window for window 0:
           Date   Open   High     Low  Close      Volume
1266 2024-08-12  71.32  76.62  69.610  75.84  28435775.0
1267 2024-08-19  75.64  77.94  73.105  76.36  28322046.0
1268 2024-08-26  76.22  78.61  73.090  77.87  23395691.0
1269 2024-09-02  76.41  76.41  67.410  67.92  30198087.0
1270 2024-09-09  68.80  72.17  66.360  71.72  27903810.0
1271 2024-09-16  70.80  74.99  68.390  69.57  31367802.0
1272 2024-09-23  69.98  75.88  69.100  74.39  24137794.0

Lab

In [ ]:
stock_predictor = LLMPredictions()
stock_predictor.load_context({})  # Manually call load_context with an empty context
result = stock_predictor.predict(event)
result